# Softmax is not enough

In [ ]:
import transformers
import torch
from torch import nn

from typing import Optional

MODEL_NAME = "google/gemma-3-1b-it"
# MODEL_NAME = "google/gemma-3n-e2b"

c:\Users\alexg\miniconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_polynomial_value(x: float, c: list[float]) -> float:
    cur_val = 0
    for i in range(len(c) - 1):
        cur_val = (cur_val + c[i]) * x
    return cur_val + c[-1]

POLY_FIT = torch.nn.Parameter(torch.tensor([-0.037, 0.481, -2.3, 4.917, -1.791]), requires_grad=True) # see Figure 6

def softmax_adaptive_temperature(logits, dim, poly_fit=POLY_FIT, dtype=torch.float32):
    """
    from "Softmax is not Enough" Figure 4, adapted to PyTorch
    """
    original_probs = torch.softmax(logits, dim=dim, dtype=dtype)
    entropy = torch.sum(-original_probs * torch.log(original_probs + 1e-9), axis=-1, keepdims=True) # compute the Shannon entropy
    beta = torch.where(
        entropy > 0.5,  # don’t overcorrect low-entropy heads
        torch.maximum(get_polynomial_value(entropy, poly_fit), torch.tensor(1.0)),  # never increase entropy
        torch.tensor(1.0)
    )
    return torch.softmax(logits * beta, dim=dim, dtype=dtype)
    
class SoftmaxZero(nn.Module):
    # for testing only
    def __init__(self):
        super(SoftmaxZero, self).__init__()

    def forward(self, logits):
        return torch.zeros_like(logits)
    
def adaptive_temperature_eager_attention_forward(
    module: nn.Module,
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    attention_mask: Optional[torch.Tensor],
    dropout: float = 0.0,
    scaling: Optional[float] = None,
    softcap: Optional[float] = None,
    **kwargs,
) -> tuple[torch.Tensor, torch.Tensor]:
    """
    A copy of gemma3's eager attention forward function, but using adaptive temperature softmax.
    """
    if scaling is None:
        scaling = module.head_dim**-0.5

    # I don't like importing a foreign module like this, but ok for now
    key_states = transformers.models.gemma3.modeling_gemma3.repeat_kv(key, module.num_key_value_groups)
    value_states = transformers.models.gemma3.modeling_gemma3.repeat_kv(value, module.num_key_value_groups)

    attn_weights = torch.matmul(query, key_states.transpose(2, 3)) * scaling

    if softcap is not None:
        attn_weights = attn_weights / softcap
        attn_weights = torch.tanh(attn_weights)
        attn_weights = attn_weights * softcap
    if attention_mask is not None:  # no matter the length, we just slice it
        causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
        attn_weights = attn_weights + causal_mask

    # upcast attention to fp32
    # attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query.dtype)
    attn_weights = softmax_adaptive_temperature(attn_weights, dim=-1, dtype=torch.float32).to(query.dtype)
    
    attn_weights = nn.functional.dropout(attn_weights, p=dropout, training=module.training)
    attn_output = torch.matmul(attn_weights, value_states)
    attn_output = attn_output.transpose(1, 2).contiguous()
    return attn_output, attn_weights

# register the new attention function
# this is so scrappy
transformers.modeling_utils.ALL_ATTENTION_FUNCTIONS.register("adaptive_temperature_eager", adaptive_temperature_eager_attention_forward)


W1020 13:00:03.911000 132796 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
adaptive_model = transformers.Gemma3ForCausalLM.from_pretrained(MODEL_NAME, attn_implementation="adaptive_temperature_eager")
original_model = transformers.Gemma3ForCausalLM.from_pretrained(MODEL_NAME, attn_implementation="eager")
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

adaptive_pipeline = transformers.pipeline("text-generation", model=adaptive_model, tokenizer=tokenizer, device="cpu")
original_pipeline = transformers.pipeline("text-generation", model=original_model, tokenizer=tokenizer, device="cpu")

c:\Users\alexg\miniconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alexg\.cache\huggingface\hub\models--google--gemma-3-1b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(

In [18]:
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a poem on Hugging Face, the company"},]
        },
    ],
]
output = original_pipeline(messages, max_new_tokens=50)
print(output[0][0]['generated_text'][-1]['content'])

Okay, here's a poem about Hugging Face, aiming to capture its essence and impact:

**The Neural Web**

A whisper of code, a digital bloom,
Hugging Face arises, dispelling gloom.
A platform vast


# CLRS Dataset Evaluation


In [46]:
# import CLRS text utilities for generating algorithmic reasoning tasks
import sys
import os
from datasets import IterableDataset, Value, Features

sys.path.append(os.path.join(os.getcwd(), 'clrs-repo'))

from clrs._src.clrs_text.huggingface_generators import clrs_generator

# Create a simple configuration with specific algorithms and fixed lengths
# This avoids the type error by using explicit numeric values
algos_and_lengths = {
    "minimum": [3, 6, 9, 12, 15]
}

ds = IterableDataset.from_generator(
        clrs_generator,
        features=Features(
            {
                "text": Value(dtype="string", id=None),
                "question": Value(dtype="string", id=None),
                "answer": Value(dtype="string", id=None),
                "algo_name": Value(dtype="string", id=None),
                "length": Value(dtype="int32", id=None),
                "use_hints": Value(dtype="bool_", id=None),
            }
        ),
        gen_kwargs={
            "algos_and_lengths": algos_and_lengths,
            # "use_hints": True
        },
    )

In [55]:
from tqdm import tqdm

def evaluate_model_on_clrs(pipeline, dataset, num_samples=10, max_new_tokens=100):
    """
    Evaluate a model on CLRS algorithmic reasoning tasks
    """
    results = []
    correct = 0
    total = 0
    
    # Create generator and take first num_samples
    sample_iter = iter(dataset)

    for i in tqdm(range(num_samples)):
        try:
            sample = next(sample_iter)
        except StopIteration:
            break
            
        text = sample['text']
        question = sample['question']
        correct_answer = sample['answer']
        algo_name = sample['algo_name']
        hints_used = sample['use_hints']
        
        
        # Generate model response
        
        prompt = [
            [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": f"You are a computer emulating minumum function. Find the smallest number in the array and ONLY GIVE its index, nothing else"},]
                },
                {
                    "role": "user",
                    "content": [{"type": "text", "text": f"{question}"},]
                },
            ],
        ]

        output = pipeline(prompt, max_new_tokens=max_new_tokens)
        model_answer = output[0][0]['generated_text'][-1]['content']
        
        # Simple exact match evaluation
        is_correct = correct_answer.strip().lower() == model_answer.lower()
        if is_correct:
            correct += 1
        total += 1
        
        results.append({
            
            'algo_name': algo_name,
            'question': question,
            'correct_answer': correct_answer,
            'model_answer': model_answer,
            'is_correct': is_correct
        })
        
        print(f"\n--- Example {i+1} ({algo_name}) ---")
        # print(f"Text: {text}...")
        print(f"Question: {question}...")
        print(f"Correct: {correct_answer}")
        print(f"Model: {model_answer}...")
        # print(f"Hints Used: {hints_used}")
        print(f"Correct: {is_correct}")
    
    accuracy = correct / total if total > 0 else 0
    print(f"\nOverall Accuracy: {correct}/{total} = {accuracy:.3f}")
    
    return results, accuracy

In [56]:
# Evaluate baseline model with standard softmax
print("Evaluating BASELINE MODEL (standard softmax):")
print("=" * 50)
baseline_results, baseline_accuracy = evaluate_model_on_clrs(
    original_pipeline, ds, num_samples=20
)

Evaluating BASELINE MODEL (standard softmax):


  5%|▌         | 1/20 [00:01<00:29,  1.58s/it]


--- Example 1 (minimum) ---
Question: minimum:
key: [0.548 0.715 0.602 0.544 0.423 0.645 0.437 0.891 0.963 0.383 0.791 0.528]
min:
...
Correct: 9


Model: 0
...
Correct: False


 10%|█         | 2/20 [00:03<00:28,  1.56s/it]


--- Example 2 (minimum) ---
Question: minimum:
key: [0.568 0.925 0.071 0.087 0.02 0.832 0.778 0.87 0.978 0.799 0.461 0.78]
min:
...
Correct: 4


Model: 0
...
Correct: False


...


--- Example 18 (minimum) ---
Question: minimum:
key: [0.67 0.21 0.128 0.315 0.363 0.57 0.438 0.988 0.102 0.208 0.161 0.653 0.253 0.466 0.244]
min:
...
Correct: 8


Model: 0
...
Correct: False


 95%|█████████▌| 19/20 [00:31<00:01,  1.47s/it]


--- Example 19 (minimum) ---
Question: minimum:
key: [0.612 0.616 0.943 0.681 0.359 0.437 0.697 0.06 0.666]
min:
...
Correct: 7


Model: 0
...
Correct: False


100%|██████████| 20/20 [00:32<00:00,  1.63s/it]


--- Example 20 (minimum) ---
Question: minimum:
key: [0.158 0.11 0.656 0.138 0.196 0.368 0.82 0.097 0.837 0.096 0.976 0.468 0.976 0.604 0.739]
min:
...
Correct: 9


Model: 0
...
Correct: False

Overall Accuracy: 0/20 = 0.000


In [57]:
# Evaluate adaptive temperature model
print("\nEvaluating ADAPTIVE TEMPERATURE MODEL:")
print("=" * 50)
adaptive_results, adaptive_accuracy = evaluate_model_on_clrs(
    adaptive_pipeline, ds, num_samples=20
)


Evaluating ADAPTIVE TEMPERATURE MODEL:


  5%|▌         | 1/20 [00:05<01:39,  5.24s/it]


--- Example 1 (minimum) ---
Question: minimum:
key: [0.548 0.715 0.602 0.544 0.423 0.645 0.437 0.891 0.963 0.383 0.791 0.528]
min:
...
Correct: 9


Model: ... 7

*   **...** and then...

*   **0**
...
Correct: False


 10%|█         | 2/20 [00:11<01:47,  5.99s/it]


--- Example 2 (minimum) ---
Question: minimum:
key: [0.568 0.925 0.071 0.087 0.02 0.832 0.778 0.87 0.978 0.799 0.461 0.78]
min:
...
Correct: 4


Model: You're asking for me to write a whole paragraph. I'm ready for the next prompt!  Let's go.
...
Correct: False


...


--- Example 18 (minimum) ---
Question: minimum:
key: [0.67 0.21 0.128 0.315 0.363 0.57 0.438 0.988 0.102 0.208 0.161 0.653 0.253 0.466 0.244]
min:
...
Correct: 8


Model: ...
Correct: False


 95%|█████████▌| 19/20 [02:04<00:03,  3.79s/it]


--- Example 19 (minimum) ---
Question: minimum:
key: [0.612 0.616 0.943 0.681 0.359 0.437 0.697 0.06 0.666]
min:
...
Correct: 7


Model: ...
Correct: False


100%|██████████| 20/20 [02:10<00:00,  6.50s/it]


--- Example 20 (minimum) ---
Question: minimum:
key: [0.158 0.11 0.656 0.138 0.196 0.368 0.82 0.097 0.837 0.096 0.976 0.468 0.976 0.604 0.739]
min:
...
Correct: 9


Model: Let me know the number you want me to generate, and I'll do my best!...
Correct: False

Overall Accuracy: 0/20 = 0.000


In [58]:
# Compare results
print("\n" + "=" * 60)
print("COMPARISON RESULTS")
print("=" * 60)
print(f"Baseline (standard softmax):     {baseline_accuracy:.3f}")
print(f"Adaptive temperature softmax:    {adaptive_accuracy:.3f}")
print(f"Improvement:                     {adaptive_accuracy - baseline_accuracy:+.3f}")

# Analyze by algorithm type
from collections import defaultdict

def analyze_by_algorithm(results):
    algo_stats = defaultdict(lambda: {'correct': 0, 'total': 0})
    
    for result in results:
        algo = result['algo_name']
        algo_stats[algo]['total'] += 1
        if result['is_correct']:
            algo_stats[algo]['correct'] += 1
    
    return {algo: stats['correct']/stats['total'] if stats['total'] > 0 else 0 
            for algo, stats in algo_stats.items()}

baseline_by_algo = analyze_by_algorithm(baseline_results)
adaptive_by_algo = analyze_by_algorithm(adaptive_results)

print(f"\nPer-algorithm performance:")
print(f"{'Algorithm':<20} {'Baseline':<10} {'Adaptive':<10} {'Diff':<10}")
print("-" * 50)

all_algos = set(baseline_by_algo.keys()) | set(adaptive_by_algo.keys())
for algo in sorted(all_algos):
    baseline_acc = baseline_by_algo.get(algo, 0)
    adaptive_acc = adaptive_by_algo.get(algo, 0)
    diff = adaptive_acc - baseline_acc
    print(f"{algo:<20} {baseline_acc:<10.3f} {adaptive_acc:<10.3f} {diff:<+10.3f}")


COMPARISON RESULTS
Baseline (standard softmax):     0.000
Adaptive temperature softmax:    0.000
Improvement:                     +0.000

Per-algorithm performance:
Algorithm            Baseline   Adaptive   Diff      
--------------------------------------------------
minimum              0.000      0.000      +0.000    
